In [20]:
from ase.build import nanotube
from ase.visualize import view
from ase.io import write

In [22]:
cnt = nanotube(6, 0,length=4, bond=1.42)
cnt.set_pbc((1, 0, 0))
view(cnt, viewer='x3d')

In [ ]:
# Save the coordinates to a file
write('cnt_(6,0).xyz', cnt)

### Apply strain to atoms

In [25]:
# Apply strain to atoms with z > midpoint
positions = cnt.get_positions()
z_coords = positions[:, 2]
z_mid = (z_coords.max() + z_coords.min()) / 2

# Stretch top half by 5%
for i, z in enumerate(z_coords):
    if z > z_mid:
        positions[i, 2] *= 1.05  # apply 5% elongation in z

cnt.set_positions(positions)
view(cnt, viewer='x3d')


In [ ]:
# Save to file
write('cnt_strained.xyz', cnt)